한 행에 시계열에 대한 데이터가 다 들어가야한다. <br/>
컬럼3, 6, 9, ~ 240
한 행의 답이 20개의 답이 되어줄 것이다.


In [1]:
%pip install catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb  # LightGBM
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")


Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv("./data/rainfall_train.csv", encoding='utf-8', index_col=False)
df['prob_0'] = 100 - df['rainfall_train.v01']
df['prob_1'] = df['rainfall_train.v01'] - df['rainfall_train.v02']
df['prob_2'] = df['rainfall_train.v02'] - df['rainfall_train.v03']
df['prob_3'] = df['rainfall_train.v03'] - df['rainfall_train.v04']
df['prob_4'] = df['rainfall_train.v04'] - df['rainfall_train.v05']
df['prob_5'] = df['rainfall_train.v05'] - df['rainfall_train.v06']
df['prob_6'] = df['rainfall_train.v06'] - df['rainfall_train.v07']
df['prob_7'] = df['rainfall_train.v07'] - df['rainfall_train.v08']
df['prob_8'] = df['rainfall_train.v08'] - df['rainfall_train.v09']
df['prob_9'] = df['rainfall_train.v09']
df.drop(columns=['rainfall_train.v01',
                 'rainfall_train.v02',
                 'rainfall_train.v03',
                 'rainfall_train.v04',
                 'rainfall_train.v05',
                 'rainfall_train.v06',
                 'rainfall_train.v07',
                 'rainfall_train.v08',
                 'rainfall_train.v09'], inplace=True)
df = df[df['rainfall_train.vv'] >= 0]
df["expected"] = (df["prob_0"] * 0\
                +df["prob_1"] * 10\
                +df["prob_2"] * 20\
                +df["prob_3"] * 30\
                +df["prob_4"] * 40\
                +df["prob_5"] * 50\
                +df["prob_6"] * 60\
                +df["prob_7"] * 70\
                +df["prob_8"] * 80\
                +df["prob_9"] * 90) / 100
df.drop(columns=["prob_0","prob_1","prob_2","prob_3","prob_4","prob_5","prob_6","prob_7","prob_8","prob_9",
                 'Unnamed: 0', 'rainfall_train.fc_year',	'rainfall_train.fc_month',	'rainfall_train.fc_day',	'rainfall_train.fc_hour'
                 ], inplace=True)
df

,rainfall_train.stn4contest,rainfall_train.dh,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.vv,rainfall_train.class_interval,expected
0,STN001,3,A,5,1,12,0.0,0,0.0
1,STN001,6,A,5,1,15,0.0,0,0.0
2,STN001,9,A,5,1,18,0.0,0,0.0
3,STN001,12,A,5,1,21,0.0,0,0.0
4,STN001,15,A,5,2,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...
1457247,STN020,228,C,10,10,9,0.5,3,2.0
1457248,STN020,231,C,10,10,12,0.0,0,3.1
1457249,STN020,234,C,10,10,15,1.9,4,3.1
1457250,STN020,237,C,10,10,18,1.9,4,2.2


In [7]:
df["index"] = df["rainfall_train.dh"] // 12
df["index"].value_counts()
pivot_df = df.pivot_table(index=['rainfall_train.stn4contest', 
                                 'rainfall_train.ef_year', 
                                 'rainfall_train.ef_month', 
                                 'rainfall_train.ef_day', 
                                 'rainfall_train.ef_hour',
                                 'rainfall_train.vv',
                                 'rainfall_train.class_interval'
                                 ],
                          columns='index',
                          values='expected').reset_index()
pivot_df


index,rainfall_train.stn4contest,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.vv,rainfall_train.class_interval,0,1,2,...,11,12,13,14,15,16,17,18,19,20
0,STN001,A,5,1,12,0.0,0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,STN001,A,5,1,15,0.0,0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,STN001,A,5,1,18,0.0,0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,STN001,A,5,1,21,0.0,0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,STN001,A,5,2,0,0.0,0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77518,STN020,C,10,10,9,0.5,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.1
77519,STN020,C,10,10,12,0.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1,NaN
77520,STN020,C,10,10,15,1.9,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1,NaN
77521,STN020,C,10,10,18,1.9,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2,NaN


In [19]:

final_df = pivot_df.groupby([
    'rainfall_train.stn4contest', 
    'rainfall_train.ef_year', 
    'rainfall_train.ef_month', 
    'rainfall_train.ef_day', 
    'rainfall_train.ef_hour']).sum().reset_index()
final_df = pd.get_dummies(final_df, columns=['rainfall_train.stn4contest'])
final_df


,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.vv,rainfall_train.class_interval,0,1,2,3,...,rainfall_train.stn4contest_STN011,rainfall_train.stn4contest_STN012,rainfall_train.stn4contest_STN013,rainfall_train.stn4contest_STN014,rainfall_train.stn4contest_STN015,rainfall_train.stn4contest_STN016,rainfall_train.stn4contest_STN017,rainfall_train.stn4contest_STN018,rainfall_train.stn4contest_STN019,rainfall_train.stn4contest_STN020
0,A,5,1,12,0.0,0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,A,5,1,15,0.0,0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,A,5,1,18,0.0,0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,A,5,1,21,0.0,0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,A,5,2,0,0.0,0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77518,C,10,10,9,0.5,3,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
77519,C,10,10,12,0.0,0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
77520,C,10,10,15,1.9,4,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
77521,C,10,10,18,1.9,4,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True


In [11]:
final_df[
    (final_df["rainfall_train.stn4contest"] == 'STN001') &
   	(final_df["rainfall_train.ef_year"] == 'A') &	
    (final_df["rainfall_train.ef_month"] == 8) & 
    (final_df["rainfall_train.ef_day"] == 1) &
    (final_df["rainfall_train.ef_hour"] == 15)
       ]

index,rainfall_train.stn4contest,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.vv,rainfall_train.class_interval,0,1,2,...,11,12,13,14,15,16,17,18,19,20
732,STN001,A,8,1,15,21.3,8,59.2,54.9,50.2,...,37.9,37.6,41.8,43.6,40.5,38.5,30.8,30.8,31.2,0.0


In [13]:
df[(df["rainfall_train.stn4contest"] == 'STN001') &
   	(df["rainfall_train.ef_year"] == 'A') &	
    (df["rainfall_train.ef_month"] == 8) & 
    (df["rainfall_train.ef_day"] == 1) &
    (df["rainfall_train.ef_hour"] == 15)
       ]

,rainfall_train.stn4contest,rainfall_train.dh,rainfall_train.ef_year,rainfall_train.ef_month,rainfall_train.ef_day,rainfall_train.ef_hour,rainfall_train.vv,rainfall_train.class_interval,expected,index
13275,STN001,234,A,8,1,15,21.3,8,31.2,19
13351,STN001,222,A,8,1,15,21.3,8,30.8,18
13427,STN001,210,A,8,1,15,21.3,8,30.8,17
13503,STN001,198,A,8,1,15,21.3,8,38.5,16
13579,STN001,186,A,8,1,15,21.3,8,40.5,15
13655,STN001,174,A,8,1,15,21.3,8,43.6,14
13731,STN001,162,A,8,1,15,21.3,8,41.8,13
13807,STN001,150,A,8,1,15,21.3,8,37.6,12
13883,STN001,138,A,8,1,15,21.3,8,37.9,11
13959,STN001,126,A,8,1,15,21.3,8,39.7,10


In [14]:
final_df.columns

Index([   'rainfall_train.stn4contest',        'rainfall_train.ef_year',
             'rainfall_train.ef_month',         'rainfall_train.ef_day',
              'rainfall_train.ef_hour',             'rainfall_train.vv',
       'rainfall_train.class_interval',                               0,
                                     1,                               2,
                                     3,                               4,
                                     5,                               6,
                                     7,                               8,
                                     9,                              10,
                                    11,                              12,
                                    13,                              14,
                                    15,                              16,
                                    17,                              18,
                                    19,            

In [16]:
final_df.to_csv("./data/pivot_expected.csv", encoding='utf-8', index=False)

In [17]:

X = final_df.drop(columns=[
    'rainfall_train.st',
    'rainfall_train.vv',
    'rainfall_train.class_interval'
])
y = final_df[['rainfall_train.class_interval']]

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42) 

# 개별 모델 학습
catboost = CatBoostClassifier(iterations=100, random_seed=42, silent=True)
lgbm = LGBMClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(n_estimators=100, random_state=42)

catboost.fit(X_train, y_train)
lgbm.fit(X_train, y_train)
xgb.fit(X_train, y_train)

# 검증 데이터로 개별 모델 예측 (확률 값 사용)
catboost_val_pred_proba = catboost.predict_proba(X_val)
lgbm_val_pred_proba = lgbm.predict_proba(X_val)
xgb_val_pred_proba = xgb.predict_proba(X_val)

# Stacked features for meta model (확률 값 사용)
stacked_features_proba = np.hstack((catboost_val_pred_proba, lgbm_val_pred_proba, xgb_val_pred_proba))
meta_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
meta_model.fit(stacked_features_proba, y_val)

catboost_test_pred_proba = catboost.predict_proba(X_test)
lgbm_test_pred_proba = lgbm.predict_proba(X_test)
xgb_test_pred_proba = xgb.predict_proba(X_test)

stacked_test_features_proba = np.hstack((catboost_test_pred_proba, lgbm_test_pred_proba, xgb_test_pred_proba))
final_pred = meta_model.predict(stacked_test_features_proba)
results_df = pd.DataFrame({'estimated_rank': final_pred.ravel(), 'rainfall_train.class_interval': np.array(y_test).ravel()})

# H, F, M, C 계산 함수
def calculate_csi_components(df):
    H = ((df['estimated_rank'] != 0) & (df['rainfall_train.class_interval'] != 0) & (df['estimated_rank'] == df['rainfall_train.class_interval'])).sum()
    F = ((df['estimated_rank'] != 0) & (df['estimated_rank'] != df['rainfall_train.class_interval'])).sum()
    M = ((df['estimated_rank'] == 0) & (df['rainfall_train.class_interval'] != 0)).sum()
    C = ((df['estimated_rank'] == 0) & (df['rainfall_train.class_interval'] == 0)).sum()
    return H, F, M, C

# CSI 계산 함수
def calculate_csi(H, F, M):
    return H / (H + F + M) if (H + F + M) > 0 else 0

# H, F, M, C 계산
H, F, M, C = calculate_csi_components(results_df)

# CSI 계산
csi_value = calculate_csi(H, F, M)
print(f"CSI: {csi_value}")
print(f"H: {H}, F: {F}, M: {M}, C: {C}")

# H, F, M, C 값 시각화
metrics = pd.DataFrame({'Metric': ['H', 'F', 'M', 'C'], 'Count': [H, F, M, C]})

plt.figure(figsize=(10, 6))
sns.barplot(x='Metric', y='Count', data=metrics, palette='viridis')
plt.title('H, F, M, C Counts')
plt.xlabel('Metric')
plt.ylabel('Count')
plt.show()



CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="STN016": Cannot convert 'b'STN016'' to float